# Extracción de producción contador SIMEL

Los datos de energia producida provienen del contador fiscal de la instalación. La adquisición de este dato se realiza por parte de **Axpo** como representante de mercado, haciendo las veces de concentrador de medida de su cartera de clientes. Esta lectura se realiza de forma diaria por parte de **SIMEL** organo encargado de adquirir las medidas fiscales de contador de las unidades de generacion.
Axpo de forma diaria consulta mediante API esta información con los certificados necesarios y se encarga de almacenar la información en una base de datos SQL interna.

Mediante la siguiente Query se obtendran las medidas de contador. Se obtiene un valor de Energía por cada periodo en el que esta dividido el dia.

La variable UFI que es el identificador de la instalación se sustituye por un Alias para preservar la identidad de la planta.

~~~~sql
    DECLARE @UFISICA as varchar(15)='SOLABE1'
    DECLARE  @FECHASESION as varchar(8)= CONVERT(NVARCHAR(50),'20200101',112)
    DECLARE @FECHASESION2 as varchar(8)= CONVERT(NVARCHAR(50),'20240101',112)

    SELECT
	    'SOLAR' as codUFisica,
	    [PERIODO] as Period,
	    [MEDIDA]/1000 as Energy_SIMEL,
	    fecha as Fecha,
	    DATEPART(YY,fecha) as Year,
	    DATEPART(mm,fecha) as Month,
	    DATEPART(dd,fecha) as Day,
    FROM [EGL_READ].[dbo].[LIQ_MEDIDAS_SIMEL]
    WHERE unidad = @UFISICA
    AND fecha >= @FECHASESION
    AND fecha <= @FECHASESION2
    order by Fecha, Period asc



El resultado de esta query se exporta en archivo csv separado por comas

## 1.Importado de librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 2.Carga de datos desde fichero externo (csv)

In [23]:
from google.colab import drive
drive.mount('/content/drive')
SIMEL=pd.read_csv("/content/drive/MyDrive/TFM_Kschool/2.RECOPILACION_DE_DATOS/2.2_REPOSITORIO_DATOS_BRUTOS/ENERGIA_BRUTO1.csv",sep=";")
SIMEL.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,codUFisica,Period,Energy_SIMEL,Fecha,Year,Month,Day
0,SOLAR,1,0.0,2021-01-01 00:00:00,2021,1,1
1,SOLAR,2,0.0,2021-01-01 00:00:00,2021,1,1
2,SOLAR,3,0.0,2021-01-01 00:00:00,2021,1,1
3,SOLAR,4,0.0,2021-01-01 00:00:00,2021,1,1
4,SOLAR,5,0.0,2021-01-01 00:00:00,2021,1,1


##3.Procesado y limpieza del dato

Comprobamos el tipo de dato del csv para evaluar si es necesario una conversión del tipo de dato.

In [24]:
SIMEL.dtypes

codUFisica       object
Period            int64
Energy_SIMEL    float64
Fecha            object
Year              int64
Month             int64
Day               int64
dtype: object

Es necesario la conversion de las variables Fecha y eliminar la hora ya que siempre es 00:00.
Tampoco se van a utilizar en primera instancia las variables Year, Month, Day

In [28]:
#conversion Fecha a fecha simple
SIMEL['Fecha']
SIMEL['Fecha']=pd.to_datetime(SIMEL['Fecha'])
#SIMEL.dtypes
SIMEL=pd.DataFrame(SIMEL, columns = ['codUFisica','Fecha','Period','Energy_SIMEL'])
print(SIMEL)

      codUFisica      Fecha  Period  Energy_SIMEL
0          SOLAR 2021-01-01       1           0.0
1          SOLAR 2021-01-01       2           0.0
2          SOLAR 2021-01-01       3           0.0
3          SOLAR 2021-01-01       4           0.0
4          SOLAR 2021-01-01       5           0.0
...          ...        ...     ...           ...
25158      SOLAR 2023-11-15      20           0.0
25159      SOLAR 2023-11-15      21           0.0
25160      SOLAR 2023-11-15      22           0.0
25161      SOLAR 2023-11-15      23           0.0
25162      SOLAR 2023-11-15      24           0.0

[25163 rows x 4 columns]


Las señales provinientes de campo son sensibles a tomar valores erroneos por estar originados en sensores o equipos de medida. Por lo que nos aseguramos que los valores leidos de Energia se encuentran dentro de la normalidad. Esto es, la energia debe estar limitada superiormente por la potencia de la instalacion y siempre ser positiva.
Tambien se localizarán los NULLs si los hubiera


In [29]:
#MAX
print(SIMEL.max())
#MIN
print(SIMEL.min())

codUFisica                    SOLAR
Fecha           2023-11-15 00:00:00
Period                           25
Energy_SIMEL                   48.4
dtype: object
codUFisica                    SOLAR
Fecha           2021-01-01 00:00:00
Period                            1
Energy_SIMEL                    0.0
dtype: object


La maxima prevision cumple la maxima potencia instalada y no existen valores negativos. No hay valores fuera de escala.

In [34]:
SIMEL.isnull().sum()

codUFisica      0
Fecha           0
Period          0
Energy_SIMEL    0
dtype: int64

## 4.Exportado dato procesado

In [35]:
SIMEL.to_csv("/content/drive/MyDrive/TFM_Kschool/2.RECOPILACION_DE_DATOS/2.3_REPOSITORIO_DATOS_PROCESADOS/ENERGIA_PROCESADOS.csv",sep=";")